## Notebook pour la récupération des commentaires des utilisateurs sur IMBD via Selenium

In [9]:
#pip install selenium

In [15]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [16]:
data_bo = pd.read_csv("data_bo.csv")

In [41]:
data_bo.head()

,Titre,Réalisateur,Année,Pays De Production,Recettes,Meilleur Rang
0,Avatar,James Cameron,2009,États-Unis,2923706026,1
1,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2799439100,1
2,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2320250281,3
3,Titanic,James Cameron,1997,États-Unis,2264743305,1
4,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2071310218,3


!!! attention !!! Il faut télécharger un "chromedriver" de la même version que votre version chrome, et le mettre à coté de ce fichier jupyter dans les fichiers.

In [11]:
def scrape_imdb_reviews(url, num_iterations=3, wait_time=20):
    # Initialise le navigateur
    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, wait_time)
    
    #cherche le bouton "Hide Spoilers"
    spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
    # Cliquez sur le bouton
    spoiler_button.click()
    
    # Clique sur le bouton "Load More" plusieurs fois
    for i in range(num_iterations):
        load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
        load_more_button.click()
        driver.implicitly_wait(3)

    time.sleep(3)

    # Trouve les commentaires par classe CSS
    reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

    # Imprime les commentaires pour débugger (les trois premiers commentaires)
    
    reviews_txt = []
    for review in reviews:
        reviews_txt.append(review.text)
        
    #check
    print(reviews_txt[:2])  

    # Imprime le nombre total de commentaires récupérés
    print(f"Nombre total de commentaires : {len(reviews)}")

    time.sleep(5)

    # Ferme le navigateur
    driver.quit()
    
    return reviews_txt

Les commentaires cachés avec "warning spoil" sont maintenant vides dans notre liste, contrairement à avant avec la méthode beautiful soupe.

Le lien "hidespoil"ci-dessous est ce lien:https://www.imdb.com/title/tt0499549/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0
où l'on a coché la case "hide spoiler" ce qui retire automatiquement de la liste les commentaires avec la banière warning spoil. Ce qui nous permet de ne pas avoir d'élément vide dans notre liste. 
J'ai donc modifié la fonction pour que le robot clique automatiquement sur le bouton "hide spoilers" ce qui permet d'aller vers le 2ème lien à partir du 1er lien automatiquement.

In [12]:
url_avatar = "https://www.imdb.com/title/tt0499549/reviews/?ref_=tt_ov_rt"
url_avatar_hidespoil = "https://www.imdb.com/title/tt0499549/reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0"
# Utilisation de la fonction
reviews_avatar = scrape_imdb_reviews(url_avatar, num_iterations=3, wait_time=20)

["I was waiting for this day for the longest time. I was a kid back in 2009 when this movie released. So never got to watch it back then. But now when it rereleased I booked the first day show to a IMAX show and oh boy was I blown away! This is nothing short of a masterpiece! It's beyond belief how a film like this could've been made. Every scene, every shot is perfection. You are transferred to a different world and become so engrossed in the film. Never have I attended a movie where at the end of it people got up from their seats and started clapping! Last time this happened was after Infinity War. That movie too was a damn brilliant one. I'm from India and this is legit rare, where people go crazy, so crazy after any English film. This was one experience that I will never forget. I'm definitely going for it again next weekend cuz just once was not enough.\n\nEvery human on the planet needs to experience this magnificent work of art!", "Forgive me, I'm going to jump from professional

Les commentaires cachés avec "warning spoil" sont maintenant vides dans notre liste, contrairement à avant avec la méthode beautiful soupe.

In [13]:
#vérification que nous avons pas de commentaire "vide"
X = np.array([len(review) for review in reviews_avatar])
X = [X==0]
np.mean(X)

0.0

In [46]:
#Test pour un autre film
url_cars = "https://www.imdb.com/title/tt0317219/reviews/?ref_=tt_ov_rt"
reviews_cars = scrape_imdb_reviews(url_cars, num_iterations=3, wait_time=20)

['"Cars didn\'t ride on it to make great time; they rode on it to have a great time."\n\nIt\'s not hard to make a successful movie. It\'s simple, really. Exceed my expectations. Make me feel. Force me to care. Deliver a somewhat clichéd message, but deliver it in such a way that the meaning resounds. Teach me the same lessons that your characters learn. And above all, entertain.\n\nPretty easy, right? Well, at least Pixar makes it look that way because with Cars they have once again succeeded.\n\nI\'ll be honest; I had my doubts about the movie. How would they be able to take a story featuring nothing but vehicles, with nary a human in sight, and keep my interest for a full two hours? Animals are one thing, but could Pixar successfully master the personification of modes of transportation?\n\nYes, they could, and they did.\n\nThanks to Director Lasseter\'s strong attention to detail, going so far as to insist that the vehicles bend and gesture in ways that were true to their constructi

In [74]:
def google_search(word, wait_time=10):
    
    # Initialise le navigateur
    driver = webdriver.Chrome()
    
    # Ouvrir Google
    driver.get("https://www.google.com")
    wait = WebDriverWait(driver, wait_time)
    
    # Attendre que le bouton soit présent

    button = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='QS5gu sy4vM' and text()='Tout refuser']")))

     # Cliquez sur le bouton
    button.click()
    wait = WebDriverWait(driver, wait_time)
    
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys("imdb user review "+word)
    search_box.send_keys(Keys.RETURN)
    
    # Attente que les résultats de recherche soient chargés
    wait = WebDriverWait(driver, wait_time)
    wait.until(EC.presence_of_element_located((By.ID, "search")))

    # Trouver le premier lien dans les résultats de recherche et cliquer dessus
    first_link = driver.find_element(By.CSS_SELECTOR, "h3")
    first_link.click()

    # Attendre quelques secondes pour voir le résultat (à adapter selon votre besoin)
    driver.implicitly_wait(5)

    # Fermer le navigateur
    driver.quit()
    
# Exemple d'utilisation
google_search("Avatar")

In [71]:
def google_search(word, wait_time=10):
    # Initialise le navigateur
    driver = webdriver.Chrome()

    # Ouvrir Google
    driver.get("https://www.google.com")
    wait = WebDriverWait(driver, wait_time)

    # Attendre que le bouton soit présent
    button = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='QS5gu sy4vM' and text()='Tout refuser']")))

    # Cliquez sur le bouton
    button.click()
    wait = WebDriverWait(driver, 20)

    # Actualiser la page pour que le cookie prenne effet
    driver.refresh()

    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys("imdb user review " + word)
    search_box.send_keys(Keys.RETURN)

    # Attente que les résultats de recherche soient chargés
    wait = WebDriverWait(driver, wait_time)
    wait.until(EC.presence_of_element_located((By.ID, "search")))

    # Trouver le premier lien dans les résultats de recherche et cliquer dessus
    wait = WebDriverWait(driver, wait_time)
    first_link = driver.find_element(By.CSS_SELECTOR, "a")
    wait = WebDriverWait(driver, wait_time)
    first_link.click()

    # Attendre quelques secondes pour voir le résultat (à adapter selon votre besoin)
    driver.implicitly_wait(5)

    # Fermer le navigateur
    driver.quit()

# Exemple d'utilisation
google_search("Harry Potter")


ElementClickInterceptedException: Message: element click intercepted: Element <a jsname="BKxS1e" class="gyPpGe" role="link" tabindex="0" jsaction="i3viod" data-ved="0ahUKEwjRgLzL19KCAxWVRaQEHaTmCR4Q67oDCAU">...</a> is not clickable at point (65, 80). Other element would receive the click: <div class="dodTBe" id="sfcnt"></div>
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00B772A3+45731]
	(No symbol) [0x00B02D51]
	(No symbol) [0x009F880D]
	(No symbol) [0x00A307AB]
	(No symbol) [0x00A2F2AF]
	(No symbol) [0x00A2D89B]
	(No symbol) [0x00A2CBA1]
	(No symbol) [0x00A24FC0]
	(No symbol) [0x00A47FDC]
	(No symbol) [0x00A24A4E]
	(No symbol) [0x00A48254]
	(No symbol) [0x00A5B7A2]
	(No symbol) [0x00A47DD6]
	(No symbol) [0x00A231F6]
	(No symbol) [0x00A2439D]
	GetHandleVerifier [0x00E80716+3229462]
	GetHandleVerifier [0x00EC84C8+3523784]
	GetHandleVerifier [0x00EC214C+3498316]
	GetHandleVerifier [0x00C01680+611968]
	(No symbol) [0x00B0CCCC]
	(No symbol) [0x00B08DF8]
	(No sy

In [69]:
google_search("Harry Potter")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00B772A3+45731]
	(No symbol) [0x00B02D51]
	(No symbol) [0x009F86D0]
	(No symbol) [0x00A2CAC4]
	(No symbol) [0x00A24FC0]
	(No symbol) [0x00A47FDC]
	(No symbol) [0x00A24A4E]
	(No symbol) [0x00A48254]
	(No symbol) [0x00A5B7A2]
	(No symbol) [0x00A47DD6]
	(No symbol) [0x00A231F6]
	(No symbol) [0x00A2439D]
	GetHandleVerifier [0x00E80716+3229462]
	GetHandleVerifier [0x00EC84C8+3523784]
	GetHandleVerifier [0x00EC214C+3498316]
	GetHandleVerifier [0x00C01680+611968]
	(No symbol) [0x00B0CCCC]
	(No symbol) [0x00B08DF8]
	(No symbol) [0x00B08F1D]
	(No symbol) [0x00AFB2C7]
	BaseThreadInitThunk [0x76C57BA9+25]
	RtlInitializeExceptionChain [0x779CBD3B+107]
	RtlClearBits [0x779CBCBF+191]


In [93]:
def scrap_review_all(movies, wait_time=10, num_iterations=3):
    # Initialise le navigateur
    driver = webdriver.Chrome()
    dico_all_comments = {}
    
    for movie in movies:
        # Ouvrir Google
        driver.get("https://www.google.com")
        wait = WebDriverWait(driver, wait_time)
        
        # Attendre que le bouton soit présent
        button = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='QS5gu sy4vM' and text()='Tout refuser']")))
        
        # Cliquez sur le bouton
        button.click()
        
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys("imdb  " + movie + " reviews")
        search_box.send_keys(Keys.RETURN)
        
        # Attente que les résultats de recherche soient chargés
        wait.until(EC.presence_of_element_located((By.ID, "search")))
        
        time.sleep(2)
        
        # Trouver le premier lien dans les résultats de recherche et cliquer dessus
        first_link = driver.find_element(By.CSS_SELECTOR, "h3")
        first_link.click()
        
        time.sleep(2)
        
        wait = WebDriverWait(driver, wait_time)
        
        user_comments_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))

         # Cliquez sur le bouton
        user_comments_button.click()
        
        wait = WebDriverWait(driver, wait_time)
    
        # Cherche le bouton "Hide Spoilers"
        spoiler_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='faceter-facets-text' and text()='Hide Spoilers']")))
        
        # Cliquez sur le bouton "Hide Spoilers"
        spoiler_button.click()
    
        # Clique sur le bouton "Load More" plusieurs fois
        for i in range(num_iterations):
            load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
            load_more_button.click()
            driver.implicitly_wait(3)
            time.sleep(2)

        time.sleep(2)

        # Trouve les commentaires par classe CSS
        reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

        
        reviews_txt = [review.text for review in reviews]
        dico_all_comments[movie] = reviews_txt
        
        time.sleep(2)
    
    return dico_all_comments

In [95]:
comments_all = scrap_review_all(list(data_bo.iloc[:5,0]), wait_time=20)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00B772A3+45731]
	(No symbol) [0x00B02D51]
	(No symbol) [0x009F880D]
	(No symbol) [0x00A2B940]
	(No symbol) [0x00A2BE0B]
	(No symbol) [0x00A5D1F2]
	(No symbol) [0x00A48024]
	(No symbol) [0x00A5B7A2]
	(No symbol) [0x00A47DD6]
	(No symbol) [0x00A231F6]
	(No symbol) [0x00A2439D]
	GetHandleVerifier [0x00E80716+3229462]
	GetHandleVerifier [0x00EC84C8+3523784]
	GetHandleVerifier [0x00EC214C+3498316]
	GetHandleVerifier [0x00C01680+611968]
	(No symbol) [0x00B0CCCC]
	(No symbol) [0x00B08DF8]
	(No symbol) [0x00B08F1D]
	(No symbol) [0x00AFB2C7]
	BaseThreadInitThunk [0x76C57BA9+25]
	RtlInitializeExceptionChain [0x779CBD3B+107]
	RtlClearBits [0x779CBCBF+191]


In [52]:
list(data_bo.iloc[5,0])

['A',
 'v',
 'e',
 'n',
 'g',
 'e',
 'r',
 's',
 ':',
 ' ',
 'I',
 'n',
 'f',
 'i',
 'n',
 'i',
 't',
 'y',
 ' ',
 'W',
 'a',
 'r']